<a href="https://colab.research.google.com/github/andradejmn/Evapo_Global/blob/main/Evapo_global_S_SEBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install geemap
import subprocess
try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

import ee
import geemap
import os
#Map = geemap.Map(center=[-22.7,-50.4], zoom=8)
#ee.Authenticate()

Installing geemap ...


In [2]:
#import hydrostats.metrics as hm
import numpy as np
import pandas as pd
import datetime
import pprint
import math
import matplotlib.pyplot as plt

In [3]:
#Processamento e visualização do Geopandas 
# Important library for many geopython libraries
#!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
#!pip install folium
# Install plotlyExpress
#!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 37 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-pkg-resources all 39.0.1-2 [98.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 k

In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
#import matplotlib
#import matplotlib.pyplot as plt 
#import folium
#import plotly_express as px

In [5]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=V6tYHFriI70TymOsApPJjGgu4MEQaquNUr-y9C7YpUQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhGKVCR5s-ly8X8BoYKP0zE-xtOwJ_E11PvSo7rGXr09imCkugpQmA

Successfully saved authorization token.


In [6]:
#in_csv = 'https://github.com/andradejmn/Evapo_Global/blob/main/Coordenadas_Torres.csv'
in_csv = '/content/Coordenadas_Torres.csv'

In [7]:
df = pd.read_csv(in_csv)
df.head()
df_2=(df.iloc[0, :])
#print(df.iloc[0, 0:2])
print(df.iloc[0, :])
print(df_2)
#df[df.ID="AU-ASM"]

FileNotFoundError: ignored

In [ ]:
df_2=df.loc [df ['ID']=="AU-ASM"]

In [ ]:
df_2.Time_end[0]

In [ ]:
Time_star=df_2.Time_end[0]
Time_edn=df_2.Time_end[0]

In [ ]:
#print(df[student.ID=70])

In [ ]:
fc = geemap.pandas_to_ee(df_2, latitude="Latitude", longitude="Longitude")

In [ ]:
Map = geemap.Map(center=[-22.7,-50.4], zoom=8)
#Map

In [ ]:
Brasil = ee.Geometry.Rectangle(-75,5,-30,-40)
Map.addLayer(Brasil, {}, "pandas to ee")
Map.addLayer(fc, {}, "pandas to ee")
Map

###Codigo de Evapotranspiração 


In [ ]:
#----------------------Analise na torre ------------------------------------------------------------------
#Entre com o ano! //
# Dados Globais 
#Time_star=df_2.Time_end[0]
#Time_edn=df_2.Time_end[0]
startDate =Time_star 
endDate = Time_edn

pts =fc
#pts.

In [ ]:
df_2.Latitude[0]
df_2.Longitude[0]

In [ ]:
pts = ee.Geometry.Point([df_2.Latitude[0], df_2.Longitude[0]])

In [ ]:
#----------------------região de interesse ------------------------------------------------------------------
#pts_Center= pts.centroid() #Coleta do Centroide da AI 
#print(AT_shape)
#AT=AT_Center.buffer(5000) #30 Km de Raio 
roi = pts.buffer(150000).bounds(); #Testar melhor buffer.
Map.addLayer(roi,{},'roi')
Map.centerObject(roi, 6)

# Para clicar IC
def clipper(image):
  return image.clip(roi)


In [ ]:
#----------------------funcoes ------------------------------------------------------------------
# Para clicar IC
def clipper(image):
  return image.clip(roi)

# Para setar date
def dates(img):
  date = img.date().format('YYYY-MM-dd')
  return img.set('date', date)

# Para scale dos produtos
def scale1(scene):
  scl = scene.multiply(0.0001)
  return scl.set('system:time_start',scene.get('system:time_start'))

def scale2(scene):
  scl = scene.multiply(0.02)
  return scl.set('system:time_start',scene.get('system:time_start'))

# Para renomear LST
def renome_terra(image):
  return image.rename('LST_Terra')

def renome_aqua(image):
  return image.rename('LST_Aqua')

# Para calcular indices biofisicos
def calc_IB(image):
  b1 = image.select(['Nadir_Reflectance_Band1'])
  b2 = image.select(['Nadir_Reflectance_Band2'])
  b3 = image.select(['Nadir_Reflectance_Band3'])
  b4 = image.select(['Nadir_Reflectance_Band4'])
  b5 = image.select(['Nadir_Reflectance_Band5'])
  b6 = image.select(['Nadir_Reflectance_Band6'])
  b7 = image.select(['Nadir_Reflectance_Band7'])

  albedo = (b1.multiply(0.215)).add(b2.multiply(0.266)) \
  .add(b3.multiply(0.242)).add(b4.multiply(0.129)) \
  .add(b5.multiply(0.000)).add(b6.multiply(0.112)) \
  .add(b7.multiply(0.036)).rename('albedo')

  ndvi = (b2.subtract(b1)).divide(b2.add(b1)).rename('ndvi')

  img_1 = ee.Image.constant(0).toFloat().rename('img_1')

  return image.set('system:time_start',image.get('system:time_start')) \
  .addBands(ee.Image(albedo).float()) \
  .addBands(ee.Image(ndvi).float()) \
  .addBands(ee.Image(img_1)) \
  .clip(roi)

# Para unir conjunto de dados
filterTimeEq = ee.Filter.equals({
  'leftField': 'date',
  'rightField': 'date'
})
def cat(feature):
  return ee.Image.cat(feature.get('primary'), feature.get('secondary'))

In [ ]:
#----------------------Lista de coeficientes ------------------------------------------------------------------
# conversão Kelvin to Celsius
c3 = ee.Number(273.15)
# Von karman constant
von = ee.Number(0.41)
#Stefan-Boltzmann constant
sigma_SB = ee.Number(0.000000056704)

#----------------------Dados de entrada------------------------------------------------------------------
# Dados de reflectancia
dataset_MCD43A4 = ee.ImageCollection('MODIS/006/MCD43A4') \
                  .filter(ee.Filter.date(startDate, endDate)) \
                  .map(clipper) \
                  .map(dates)

#print(dataset_MCD43A4)

# Temperatura de superficie
# Terra
LST = ee.ImageCollection('MODIS/006/MOD11A1') \
          .filter(ee.Filter.date(startDate, endDate)) \
          .select('LST_Day_1km') \
          .map(scale2) \
          .map(clipper) \
          .map(dates) \
          .map(renome_terra)
# print(LST)
# Map.addLayer(LST,{},'LST')

# Aqua
LST1 = ee.ImageCollection("MODIS/006/MYD11A1") \
  .filter(ee.Filter.date(startDate, endDate)) \
          .select('LST_Day_1km') \
          .map(scale2) \
          .map(clipper) \
          .map(dates) \
          .map(renome_aqua)
# print(LST1)
# Map.addLayer(LST1,{},'LST1')

In [ ]:
#----------------------Computo de variaveis------------------------------------------------------------------
reflec_nadir = dataset_MCD43A4.map(scale1).map(dates)

IndexBiofisico = (reflec_nadir.map(calc_IB)).select(['albedo','ndvi','img_1'])
#print(IndexBiofisico,'IndexBiofisico')

Data_set_combineLST = ee.ImageCollection(ee.Join.inner().apply(LST, LST1, filterTimeEq)).set('foo', {})
Data_set_combineLST = Data_set_combineLST.map(cat)
# print(Data_set_combineLST,'Data_set_combineLST')

Data_set_combineLST_IB = ee.ImageCollection(ee.Join.inner().apply(IndexBiofisico, Data_set_combineLST, filterTimeEq))
Data_set_combineLST_IB = Data_set_combineLST_IB.map(cat)
#print(Data_set_combineLST_IB,'Data_set_combineLST_IB')

In [ ]:
#----------------------Seleção pixels frio e quente------------------------------------------------------------------
# quente
def calc_pq(image):
  albedo = image.select(['albedo'])
  ndvi = image.select(['ndvi'])
  lst = image.select(['LST_Terra'])

  alb_cand_pq_inf = albedo.reduceRegion(**{
    'reducer': ee.Reducer.percentile([50]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  alb_cand_pq_inf = alb_cand_pq_inf.toImage()
  alb_cand_pq_inf = alb_cand_pq_inf.clip(roi)

  alb_cand_pq_sup = albedo.reduceRegion(**{
    'reducer': ee.Reducer.percentile([75]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  alb_cand_pq_sup = alb_cand_pq_sup.toImage()
  alb_cand_pq_sup = alb_cand_pq_sup.clip(roi)

  ndvi_cand_pq_sup = ndvi.reduceRegion(**{
    'reducer': ee.Reducer.percentile([15]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  ndvi_cand_pq_sup = ndvi_cand_pq_sup.toImage()
  ndvi_cand_pq_sup = ndvi_cand_pq_sup.clip(roi)

  cand_pq_alb_ndvi = (albedo.gt(alb_cand_pq_inf).And(albedo.lt(alb_cand_pq_sup))) \
  .And((ndvi.gt(0.10).And(ndvi.lt(ndvi_cand_pq_sup))))

  lst_pq = lst.multiply(cand_pq_alb_ndvi)
  lst_pq = lst_pq.selfMask()

  lst_cand_pq_sup = lst_pq.reduceRegion(**{
    'reducer': ee.Reducer.percentile([97]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  lst_cand_pq_sup = lst_cand_pq_sup.toImage()
  lst_cand_pq_sup = lst_cand_pq_sup.clip(roi)

  lst_cand_pq_inf = lst_pq.reduceRegion(**{
    'reducer': ee.Reducer.percentile([95]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  lst_cand_pq_inf = lst_cand_pq_inf.toImage()
  lst_cand_pq_inf = lst_cand_pq_inf.clip(roi)

  cand_pq_alb_ndvi_lst = (lst.gt(lst_cand_pq_inf).And(lst.lt(lst_cand_pq_sup))).multiply(cand_pq_alb_ndvi)
  cand_pq_alb_ndvi_lst = cand_pq_alb_ndvi_lst.selfMask().rename('cand_pq')
  return image.set('system:time_start',image.get('system:time_start')).addBands(ee.Image(cand_pq_alb_ndvi_lst).float())

In [ ]:
Data_set_combineLST_IB2 = Data_set_combineLST_IB.map(calc_pq)
#print(Data_set_combineLST_IB,'Data_set_combineLST_IB')

In [ ]:
# frio
def calc_pf(image):
  albedo = image.select(['albedo'])
  ndvi = image.select(['ndvi'])
  lst = image.select(['LST_Terra'])

  alb_cand_pf_inf = albedo.reduceRegion(**{
    'reducer': ee.Reducer.percentile([25]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  alb_cand_pf_inf = alb_cand_pf_inf.toImage()
  alb_cand_pf_inf = alb_cand_pf_inf.clip(roi)

  alb_cand_pf_sup = albedo.reduceRegion(**{
    'reducer': ee.Reducer.percentile([50]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  alb_cand_pf_sup = alb_cand_pf_sup.toImage()
  alb_cand_pf_sup = alb_cand_pf_sup.clip(roi)

  ndvi_cand_pf_sup = ndvi.reduceRegion(**{
    'reducer': ee.Reducer.percentile([97]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  ndvi_cand_pf_sup = ndvi_cand_pf_sup.toImage()
  ndvi_cand_pf_sup = ndvi_cand_pf_sup.clip(roi)

  cand_pf_alb_ndvi = (albedo.gt(alb_cand_pf_inf).And(albedo.lt(alb_cand_pf_sup))) \
  .And((ndvi.gt(ndvi_cand_pf_sup).And(ndvi.lt(0.99))))

  lst_pf = lst.multiply(cand_pf_alb_ndvi)
  lst_pf = lst_pf.selfMask()

  lst_cand_pf_sup = lst_pf.reduceRegion(**{
    'reducer': ee.Reducer.percentile([10]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  lst_cand_pf_sup = lst_cand_pf_sup.toImage()
  lst_cand_pf_sup = lst_cand_pf_sup.clip(roi)

  lst_cand_pf_inf = lst_pf.reduceRegion(**{
    'reducer': ee.Reducer.percentile([0]),
    'geometry': roi,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
    })

  lst_cand_pf_inf = lst_cand_pf_inf.toImage()
  lst_cand_pf_inf = lst_cand_pf_inf.clip(roi)

  cand_pf_alb_ndvi_lst = (lst.gt(lst_cand_pf_inf).And(lst.lt(lst_cand_pf_sup))) \
  .multiply(cand_pf_alb_ndvi)

  cand_pf_alb_ndvi_lst = cand_pf_alb_ndvi_lst.selfMask().rename('cand_pf')

  return image.set('system:time_start',image.get('system:time_start')) \
  .addBands(ee.Image(cand_pf_alb_ndvi_lst).float())


In [ ]:
Data_set_combineLST_IB = Data_set_combineLST_IB.map(calc_pf)
#print(Data_set_combineLST_IB,'Data_set_combineLST_IB')

In [ ]:
#----------------------calculo FE------------------------------------------------------------------
def calc_FE(image):
  cand_pq = image.select(['cand_pq'])
  cand_pf = image.select(['cand_pf'])
  lst = image.select(['LST_Terra'])

  lst_hot_temp = (lst.multiply(cand_pq))

  lst_hot = lst_hot_temp.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': roi,
  #scale: 30,
   'bestEffort': True
  #maxPixels: 1e9
  })
  lst_hot = lst_hot.toImage().rename('lst_hot')
  lst_hot = (lst_hot.clip(roi))

  lst_cold_temp = (lst.multiply(cand_pf))

  lst_cold = lst_cold_temp.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': roi,
  #scale: 30,
   'bestEffort': True
  #maxPixels: 1e9
  })
  lst_cold = lst_cold.toImage().rename('lst_cold')
  lst_cold = (lst_cold.clip(roi))

  FE = ((lst_hot.subtract(lst)).divide((lst_hot.subtract(lst_cold)))) \
  .rename('FE')

  return image.set('system:time_start',image.get('system:time_start')) \
  .addBands(ee.Image(lst_hot).float()) \
  .addBands(ee.Image(lst_cold).float()) \
  .addBands(ee.Image(FE).float())


In [ ]:
Data_set_combineLST_IB = Data_set_combineLST_IB.map(calc_FE)
#print(Data_set_combineLST_IB,'Data_set_combineLST_IB')

In [ ]:
#----------------------Dados climaticos ERA5 land------------------------------------------------------------------
climatic_hour = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
                  .filter(ee.Filter.date(startDate, endDate)) \
                  .select(['surface_net_solar_radiation_hourly','surface_net_thermal_radiation_hourly']) \
                  .map(clipper) \
                  .map(dates)

In [ ]:
StartYear = 2014;
startDate = ee.Date.fromYMD(StartYear, 1, 1);
endDate = (startDate.advance(1,'year'));
#print(startDate)
startDate2=ee.Date(2014-12-31)
#print(startDate2)

In [ ]:
def func_xbo (dayOffset):
    start_ = startDate.advance(dayOffset, 'days')
    end_ = start_.advance(1, 'days')

    bd = start_
    ed = bd.advance(1, 'days')
    serie_diaria = climatic_hour.filterDate(bd, ed)
    climatica_day = serie_diaria.sum().divide(86400)

    return climatica_day.set('system:time_start', serie_diaria.first().get('system:time_start')) \



In [ ]:
numberOfDays = endDate.difference(ee.Date(startDate2), 'days')
#print(startDate)
climatic_daily = ee.ImageCollection(ee.List.sequence(0, numberOfDays.subtract(1))).map(func_xbo)

In [ ]:
def calc_Rn_d(image):
  convert_Rs = image.select(['surface_net_solar_radiation_hourly'])
  convert_Rl = image.select(['surface_net_thermal_radiation_hourly'])

  Rn_d = (convert_Rs.add(convert_Rl)).rename('Rn_d')

  return image.set('system:time_start',image.get('system:time_start')) \
  .addBands(ee.Image(Rn_d).float())

In [ ]:
Rn_d = climatic_daily.map(calc_Rn_d).map(dates)
#print('Rn_d:', Rn_d)

Data_set_combineLST_IB = ee.ImageCollection(ee.Join.inner().apply(Data_set_combineLST_IB, Rn_d, filterTimeEq))
Data_set_combineLST_IB = Data_set_combineLST_IB.map(cat)
#print(Data_set_combineLST_IB,'Data_set_combineLST_IB')

def calc_ET_day(image):
  Rn_24 = image.select(['Rn_d'])
  FE = image.select(['FE'])

  ET_d = FE.multiply(0.035).multiply(Rn_24).rename('ET_d')

  return image.set('system:time_start',image.get('system:time_start')) \
  .addBands(ee.Image(ET_d).float())

Data_set_combineLST_IB = Data_set_combineLST_IB.map(calc_ET_day)
#print(Data_set_combineLST_IB,'Data_set_combineLST_IB')

ET_d = Data_set_combineLST_IB.select('ET_d')
#print('ET_d:', ET_d)

In [ ]:
#ET_d.getInfo()
rn_data = (((ET_d.getInfo()['features'])[0])['properties'])

#data_gee_rn = pd.DataFrame([rn_data]).T
#(data_gee_rn.set_index(dates,inplace=True))
sim_rn = pd.to_numeric(data_gee_rn[0])

In [ ]:
#print('ET_d:', ET_d.getInfo)
